

#  深度社区发现（Deep Community Detection）

---

## 1 研究背景

---

###  社区发现任务定义

> 社区发现旨在将图中的节点划分为多个结构紧密、功能相似的子集（社区、簇），使得社区内连边密集，社区间连接稀疏。

* 形式上：给定图 $G = (V, E)$，划分 $V \rightarrow \{C_1, C_2, ..., C_k\}$
* 通常作为**无监督聚类任务**，但也可做半监督/自监督建模

---

###  应用场景

| 场景    | 应用               |
| ----- | ---------------- |
| 社交网络  | 用户兴趣社群挖掘、群体行为建模  |
| 生物网络  | 蛋白质功能模块检测、疾病基因识别 |
| 电商/推荐 | 用户群体划分、兴趣簇发现     |
| 安全分析  | 异常群体发现、攻击路径检测    |

---

###  挑战

* 高维稀疏图难以建模
* 节点属性复杂/异构
* 多尺度/动态变化
* 社区标签稀缺，难以监督学习



## 2 经典社区发现算法

---

###  2.1 模块度最大化（Modularity Maximization）

定义模块度：

$$
Q = \frac{1}{2m} \sum_{i,j} \left[ A_{ij} - \frac{d_i d_j}{2m} \right] \delta(c_i, c_j)
$$

* $Q$：图结构相较于随机图的“划分质量”指标
* Greedy、Louvain、Leiden 等基于贪心方法进行模块度优化

---

###  2.2 谱聚类（Spectral Clustering）

基于图的拉普拉斯矩阵 $L$，取前 $k$ 个特征向量进行 k-means 聚类。

---

###  2.3 随机游走方法

如 Infomap、Walktrap：

* 使用节点间游走概率定义“流动社区”
* 社区内部的节点更容易互访

---

###  局限性

* 无法利用节点属性
* 不具备端到端训练能力
* 缺乏适应性和泛化性



## 3 基于自训练的社区发现算法

---

### 核心思想

> 通过**自训练（Self-training）机制**，利用初始伪标签不断优化节点聚类表示，实现端到端社区发现。

---

### 工作流程

1. **预训练阶段**：用无监督编码器学习初始嵌入（如 GCN、GAE）
2. **伪标签生成**：基于当前嵌入进行 k-means 聚类
3. **联合优化**：使用聚类损失（如 KL 散度）和重构损失优化模型
4. **迭代训练**：不断更新伪标签与表示

---

### 代表模型

| 方法           | 特点                     |
| ------------ | ---------------------- |
| DAEGC (2019) | GCN + 聚类中心 + KL 散度     |
| M3S (2021)   | 多视图结构增强自训练             |
| GALA (2022)  | GAT+Autoencoder + 聚类优化 |
| AGCN         | 融合属性与结构，进行软分配          |

---

### 常见损失函数

* 重构损失（AE/GAE）：

  $$
  \mathcal{L}_{recon} = \| A - \sigma(ZZ^\top) \|^2
  $$
* 聚类损失（KL 散度）：

  $$
  \mathcal{L}_{cluster} = KL(P \| Q)
  $$

  * $Q$：Student’s t-distribution
  * $P$：目标分布（如自定义 sharpen 过的分布）





## 4 基于迭代优化的社区发现算法

---

### 目标

结合图嵌入与聚类，通过**交替优化策略**提升聚类性能。

---

### 优化思路

1. 固定嵌入 → 优化社区标签
2. 固定标签 → 优化嵌入表示
3. 循环迭代直到收敛

---

### 代表方法

#### ARGA / ARVGA

* Autoencoder + GCN 编码器 + 聚类损失
* 结合判别器实现正则化

#### GMM-VGAE

* 使用变分图自编码器，嵌入空间符合 GMM 分布

#### SDCN

* 联合训练 AE + GCN + 聚类模块，实现深层结构感知社区发现



## 5 基于对比学习的社区发现

---

### 核心思想

> 利用图对比学习（Graph Contrastive Learning）增强节点表征，使同一社区内节点表示更相似。

---

### 对比目标设计

* **社区内一致性**：同一社区节点构成正样本对
* **社区间分离性**：不同社区节点构成负样本对

---

### 构造视图方式

* 结构扰动（丢边/加边）
* 属性扰动（mask 特征）
* 跨视图（GNN vs MLP, 多视角融合）

---

### 代表模型

| 模型     | 特点               |
| ------ | ---------------- |
| MVGRL  | 多视图对比 + 图级表示     |
| DCRN   | 同步自训练 + 局部对比机制   |
| CONTRA | 对比约束聚类，鼓励聚类一致性   |
| GCA    | 节点层对比 + 图层对比双层约束 |

---

### 优势

* 提升鲁棒性（防噪声/不完整图）
* 提供自监督信号
* 可增强聚类边界感知能力




## 6 面向异配图的图聚类算法

---

### 异配图定义

> 图中节点属性与结构信息存在**异质性或不一致性**，即“结构相似但属性差异”、“属性接近但无边连接”。

---

### 建模挑战

| 挑战      | 说明                    |
| ------- | --------------------- |
| 异质信号不一致 | 聚类结果易偏向某一侧            |
| 噪声与冲突信息 | 属性/结构中任一信息源可能误导聚类     |
| 社区语义复杂  | 不同特征维度或结构语义主导不同社区划分标准 |

---

### 代表方法

#### AGCN

* 自适应融合属性-结构信息权重
* 自注意力机制动态调节每层关注源

#### SDCN / DCRN

* 属性、结构两路编码 → 融合聚类损失
* 使用 KL 散度对两个嵌入空间对齐

#### SUBLIME

* 多视图结构子图抽样 + 表示对比学习
* 保持结构与语义信息平衡



## 7 社区发现的端到端应用

---

###  通常采用以下流程：

1. **输入**：原始图结构 + 节点特征（结构 + 属性）
2. **表示学习模块**：图嵌入（GCN/GAT/AE/VGAE）
3. **社区划分模块**：聚类（K-means/GMM）+ 聚类损失
4. **端到端优化**：联合训练表示与聚类目标

---

### 应用实例

| 场景     | 描述            |
| ------ | ------------- |
| 社交网络分析 | 社区用户兴趣发现、传播分析 |
| 推荐系统   | 相似兴趣群体推荐/召回   |
| 生物网络   | 蛋白质-蛋白质交互模块识别 |
| 文献引用图  | 自动分类、子领域演化识别  |

---

### 多任务融合趋势

* 社区发现 + 链接预测（SDCN）
* 社区发现 + 异常检测
* 社区发现 + 分类/推荐系统

---

## 小结

| 方法    | 核心机制      | 优势      | 代表模型          |
| ----- | --------- | ------- | ------------- |
| 自训练   | 伪标签+联合优化  | 弱监督、高性能 | DAEGC, GALA   |
| 迭代优化  | 嵌入与标签交替优化 | 表达性强    | ARGA, SDCN    |
| 对比学习  | 强化嵌入一致性   | 鲁棒性高    | DCRN, CONTRA  |
| 异配图方法 | 异质融合建模    | 避免偏倚    | AGCN, SUBLIME |




## Q1：到底什么是随机游走？

### 定义（Definition）

> 随机游走是一种在图中按概率在节点间“行走”的过程，每一步从当前节点转移到某个邻居节点，转移概率由边或节点的某种权重决定。

假设有图 $G = (V, E)$，每次游走从某个起点 $v_0$ 开始，之后每一步从当前节点 $v_t$ 移动到其邻居 $v_{t+1}$，概率为：

$$
P(v_{t+1} = u \mid v_t = v) = \frac{w_{vu}}{\sum_{u' \in \mathcal{N}(v)} w_{vu'}}
$$

* $\mathcal{N}(v)$：节点 $v$ 的邻居集合
* $w_{vu}$：边的权重，若为无权图通常为 1

---

### 特性（Properties）

* 是一种**马尔可夫过程**（只与当前状态有关）
* 可以设定步长、回退概率、跳转概率（如 PageRank）
* 可以是**均匀的**（等概率）或**非均匀的**（带偏好）

---

## Q2：它在图神经网络中的应用？

### 1. 用于节点表征学习（Graph Embedding）

#### DeepWalk（2014）

* 将随机游走序列当作“图句子”，借鉴 Word2Vec 思路进行节点表示学习
* 步骤：

  1. 多个节点发起随机游走，得到多个路径序列（如长度 40，次数 10）
  2. 把每条路径当作句子，节点是词，用 Skip-Gram 训练嵌入
  3. 得到高维向量用于分类、聚类、社区发现等

#### Node2Vec（2016）

* 提出**有偏随机游走**策略，引入两个超参数 $p, q$ 控制“广度优先 / 深度优先”搜索倾向，捕捉不同图结构特征：

  * BFS：鼓励探索邻居，适合结构等价（同社区）
  * DFS：鼓励探索远处节点，适合角色等价（不同社区但功能类似）

---

### 2. 图采样（用于大图 GNN 训练）

当图规模非常大时，GNN 无法在整个图上训练，常用**随机游走采样邻居子图**：

#### GraphSAGE

* 采用固定大小的邻居采样（可视为随机游走的一种简化）

#### PinSAGE（Pinterest）

* 在图上进行多次随机游走，从节点本地 neighborhood 中采样感兴趣子图

---

### 3. 图对比学习中的视图生成

* 如 DGI、GraphCL 中，对图结构进行扰动，构造“局部子图”作为正/负样本
* 随机游走作为一种子图采样方式，可生成语义相关子视图

---

## Q3：它在强化学习中的作用？

### 1. 环境建模（探索策略）

* 在强化学习中，随机游走是一种**探索策略（exploration）**，用于代理在状态空间中尝试多种路径：

  * **Random Walk Exploration**：用于在初期探索可能性
  * **Reward Shaping + Random Walk**：鼓励代理在正向区域更频繁地随机行走

### 2. MDP 中的转移建模

* 强化学习环境通常是马尔可夫决策过程（MDP）：

  * 状态 $s_t$ → 行为 $a_t$ → 状态 $s_{t+1}$
  * 如果策略是随机的，即形成一个随机游走序列

### 3. 表征学习（RL + Graph）

* 在**图结构的 RL 环境**中（如知识图谱、迷宫图），Agent 的动作就是在图上进行“有目的的随机游走”。

---

## Q4：有哪些典型应用场景？

| 领域      | 应用实例                     |
| ------- | ------------------------ |
| 图嵌入     | DeepWalk, Node2Vec, LINE |
| 图神经网络采样 | GraphSAGE, PinSAGE       |
| 推荐系统    | 用户点击图中的随机游走模拟兴趣传播        |
| 知识图谱    | 路径增强问答（Path-Ranking）     |
| 社区发现    | 游走概率用于定义节点“社区归属概率”       |
| 强化学习    | 状态空间探索、马尔可夫过程建模          |
| 图对比学习   | 生成视图、扰动结构用于自监督           |

---

## 总结

| 内容      | 描述                                |
| ------- | --------------------------------- |
| 本质      | 图上的马尔可夫路径过程                       |
| 作用      | 抽取结构特征，采样节点邻域，构建训练样本              |
| 在 GNN 中 | DeepWalk、Node2Vec、GraphSAGE、图对比学习 |
| 在 RL 中  | 状态转移建模、探索策略、图上的动作                 |
| 应用领域    | 推荐、社交、图嵌入、问答、图学习                  |


